# Analyzing ACLs and firewall rules with Batfish

Network and security engineers are responsible for ensuring that the ACLs and firewall rules in their networks are permitting and denying traffic as intended. This task usually requires manual checking or loading rulesets onto a lab device in order to test its behavior on example packets of interest. These methods are not only time consuming but also error-prone. 

Batfish makes it easy to see what each ACL or firewall ruleset, which we generally call *filters*, will do with a given packet, right down to the line of the filter that matches it, and get guarantees on how the filter treats a large spaces of packets. Batfish also provides a sanity check to help ensure that every line in a filter matches at least some packets. 

In this notebook, we demonstrate these capabilities. In this [other notebook](Provably%20Safe%20ACL%20and%20Firewall%20Changes.ipynb), we show how Batfish can guarantee that filter changes are safe.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/analyzing-acls-and-firewall-rules?pixel&useReferer) 

## Initialization

We use a snapshot with two devices, a router with ACLs and a firewall, an example in this notebook.  When adapting this example, `SNAPSHOT_PATH` below should be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis. More example networks are available [here](https://github.com/batfish/batfish/tree/master/networks).

In [1]:
# Import packages and load questions
%run startup.py

# Initialize a network and snapshot
NETWORK_NAME = "network-example-filters"
SNAPSHOT_NAME = "current"

SNAPSHOT_PATH = "networks/example-filters/current"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'current'

## Testing how a flow is treated

The `testfilters` question of Batfish shows what filters do with a particular flow and *why*. It takes as input the details of the flow and a set of filters to test. The answer shows how the flow is treated by each filter in the set.
To specify the flow, you must provide the source IP address `srcIp` and the destination `dst` (hostname or IP address). Optionally, you can also specify the ingress interface (where the flow enters the device) and header fields such as IP protocols, ports, TCP flags, and ICMP codes ([see documentation for details](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.testfilters)). The question will fill in any unspecified fields with reasonable defaults.

The set of filters to examine can be narrowed using various criteria, including regexes for node and filter names. Left unspecified, `testfilters` will give results for every filter in the network.

### Example 1: Test if hosts in a subnet can reach the DNS server

Suppose we wanted to test if our ACL allows hosts in a subnet can reach our DNS server. This check is easily expressed. Assume that the subnet is `10.10.10.0/24` and the DNS server is at the IP address `218.8.104.58`, then the query will be as shown below, where we have picked `10.10.10.1` a representative source IP for the subnet. 

In [2]:
# Check if a host can reach the DNS server
bfq.testfilters(srcIp="10.10.10.1",
                dst="218.8.104.58",
                ipProtocol="UDP",
                dstPort=53,
                filters="acl_in").answer().frame()

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace
0,rtr-with-acl,acl_in,rtr-with-acl->[10.10.10.1:49152->218.8.104.58:53 proto: UDP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,PERMIT,32,660 permit udp 10.10.10.0/24 218.8.104.58/32 eq domain,"Flow permitted by 'extended ipv4 access-list' named 'acl_in', index 32: 660 permit udp 10.10.10.0/24 218.8.104.58/32 eq domain"


The result above shows that the flow we tested is indeed permitted by the filter 'acl_in' on device 'rtr-with-acl'. The `Flow` column shows the exact flow tested with the default values chosen by Batfish. The `Line_Number` and `Line_Content` columns show which lines led to that action, and the `Trace` column shows how the packet got to the line. In this case, the trace is not deep, but for filters with nested objects at multiple levels it can be, and the trace helps further understand the result. 

## Example 2: Test that HTTP flows cannot go from one zone to another

Now suppose we want to test that our `firewall` device blocks HTTP flows that cross the boundary from one zone to another. This behavior can be checked  check this behavior using `testfilters` using queries like the one below. This query is testing what the output filter on `GigabitEthernet0/0/0` (which is in the destination zone) treats a packet with specified header fields when it arrives at the firewall on GigabitEthernet0/0/2 (which is in the source zone).

In [3]:
# Check if a host can reach the DNS server
answer = bfq.testfilters(srcIp="10.114.64.1",
                         dst="10.114.60.10",
                         dstProtocol="http",
                         ingressInterface="GigabitEthernet0/0/2",
                         nodes="firewall",
                         filters="OUTPUTFILTERON:GigabitEthernet0/0/0").answer().frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace
0,firewall,~ZONE_OUTGOING_ACL~zone-admin~,firewallingressInterface: GigabitEthernet0/0/2vrf: None->[10.114.64.1:49152->10.114.60.10:80 proto: TCP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,None,no-match,Flow did not match ACL named '~ZONE_OUTGOING_ACL~zone-admin~'


As we can see the HTTP flow is indeed denied, and the columns in the table show the reason for that denial.

## Verifying how a (very large) space of flows is treated

While `testfilters` reasons about individual flows, the `reachfilter` question of Batfish provide comprehensive guarantees by reasoning about (potentially very large) space of flows. The space of flows is specified using source and destination prefixes (where the default prefix 0.0.0.0/0 denotes all 4,294,967,296 possibilities), a list of protocols, a range of ports, and so on. Given this space and a match condition, which can be `permit`, `deny` or `matchLine <linenum>`, `reachfilters` returns any flow within this space that match the condition. If no flow is returned, we can be assured that no flow in the entire space satisfies the condition. 

### Example 1: Verify that *all* hosts in a subnet can reach the DNS server

Assuming that our goal is same as that for the first example above for `testfilters` -- check that a subnet can reach the DNS server. While we used a representative source IP address in the query above, `reachfilter` lets us analyze the entire space. 

The query below is asking if there is *any* flow from `10.10.10.0/24` to `218.8.104.58` that is *denied* by the filter `acl_in`. Any answer to this query denotes a flow in the space that violates the desired guarantee. An empty result denotes the guarantee that no such flow exists. 

In [4]:
# Check if the subnet can reach the DNS server
bfq.reachfilter(src="10.10.10.0/24",
                dst="218.8.104.58",
                dstPorts=[53],
                ipProtocols=["udp"],
                query="deny",
                filters="acl_in").answer().frame()

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace
0,rtr-with-acl,acl_in,rtr-with-acl->[10.10.10.42:0->218.8.104.58:53 proto: UDP dscp:0 ecn:0 fragOff:0 length:0 state:NEW flags: n/a,DENY,22,460 deny udp 10.10.10.42/32 218.8.104.58/32 eq domain,"Flow denied by 'extended ipv4 access-list' named 'acl_in', index 22: 460 deny udp 10.10.10.42/32 218.8.104.58/32 eq domain"


As we can see, we did get a flow that matches the search condition and thus violates our desired guarantee. The columns carry the same information as those for `testfilters` and provide insight into the flow that was found and why it was returned. In this case, we see that an earlier line in the ACL was denying a specific address in the source prefix. Such needles in the haystack are impossible to find with tools like pen testing. 

### Example 2: Verify that no non-NFS flow can go from one zone to another

In the second example for `testfilters` we checked if `firewall` allowed HTTP flows from one zone to another, and we tested that using an example flow. Now, suppose that we wanted a stricter condition---that only NFS flows are allowed from one zone to another. With `reachfilter` we can verify this condition by searching for violations in the large space of non-NFS flows, as below.

In [5]:
# Check if any non-NFS (TCP) flow can go from one zone to another on
answer = bfq.reachfilter(src="10.114.64.0/24",
                         dst="10.114.60.0/24",
                         ipProtocols=["tcp"],
                         dstPorts=["0-2048", "2050-65356"],  # exclude NFS port, 2049
                         start="GigabitEthernet0/0/2",
                         nodes="firewall",
                         filters="OUTPUTFILTERON:GigabitEthernet0/0/0",
                         query="permit").answer().frame()
display_html(answer)

,Node,Filter_Name,Flow,Action,Line_Number,Line_Content,Trace


Since we got back an empty answer, we can be certain that no non-NFS flow can go from one zone to another. Such strong guarantees are impossible with any other tool today.

## Analyzing reachability of filter lines

When debugging or editing filters, it can be useful to confirm that every line is reachable---that is, each line can  match some packets that do not match earlier lines. Unreachable filter lines are often symptomatic of bugs and past edits that did not achieve policy intent. Even when they do not represent bugs, they represent opportunities to reduce the length of the filter. 

The `filterLineReachability` question identifies unreachable filter lines. Given no parameters, it will check every filter in the network, but its scope can be narrowed  using `filters` and `nodes` parameters (see [documentation](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.aclReachability)).

In [6]:
# Find unreachable lines in filters of rtr-with-acl
aclAns = bfq.aclReachability(nodes="rtr-with-acl").answer().frame()
display_html(aclAns)

Each line in the answer above identifies an unreachable line in a filter. Let's take a closer look at the second one. It shows that the line `670 permit ip 166.146.58.184 any` is unreachable because it is blocked by the earlier line `540 deny ip 166.146.58.184/12 any`. In column `Different_Action` the result shows that the action of the blocking line is different from that of the unreachable line, which is often a more worrisome situation than when actions are the same. 

Not shown there, but in the general case, filter lines can be independently unreachable because no packet will ever match them, and they can be unreachable because of multiple preceding lines (not just one). The `filterLineReachability` question outputs such unreachable lines as wel.

## Summary

In this notebook, we showed how you can use Batfish to 
1. Test how filters in the network treat a given flow
2. Verify that a large space of flows is treated in a certain way
3. Find lines in your filters that will never match any packet

If you found these capabilities useful, you may also want to checkout [this notebook](Provably%20Safe%20ACL%20and%20Firewall%20Changes.ipynb) that builds on these concepts to enable a workflow for *provably* safe changes to your filter.

***
### Get involved with the Batfish community

To get involved and learn more, join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). 